In [9]:
#Imports
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
#Fazendo as requisições para o download das páginas#Fazend 
#Criando função para receber uma URL e verificar se tem algum 
#Conteúdo do tipo HTML/texto 
def get_url(url):
    try:
        with closing(get(url,stream=True)) as resp:
            if response_true(resp):
                return resp.content
            else:
                return None
    except RequestException as e:
        log_error("Erro de solicitação ".format(url,str(e)))
        return None
    
#Função para imprimir os erros 
def log_error(e):
    print(e)
    
#Função que retorna true caso seja HTML
def response_true(resp):
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code==200
           and content_type is not None
           and content_type.find('html')>-1)
#URL do Adoro Cinema (Natal)
page = get_url("http://www.adorocinema.com/programacao/cinema-A0177/")

In [28]:
#Tratando o HTML bruto#Tratan 
html = BeautifulSoup(page, 'html.parser')

#Pegando a div com as informações dos filmes
movies = html.findAll("div", {"class": "hred"})

title_movie = []
horary = []
language = []
ranking = []
release = [] 

for movie in movies:
    title_movie.append(movie.find("a", {"class":"meta-title-link"}).text)
    
    # Pegamos o horário
    span = movie.findChild("span", {"class":"hours-item-value"})
    # Verificamos se o Span não é NoneType, se for, ele retorna "Não informado"
    horary.append(span.text if span is not None else "Não informado")
    
    # Pegando a linguagem do filme
    div = movie.findChild("div",{"class":"text"})
    language.append(div.text.replace("\n","") if div is not None else "Não informado")
    
    #Pegando a classificação dos usuários sobre os filmes
    rank = movie.findChild("span", {"class":"stareval-note"})
    ranking.append(rank.text.replace("\n","") if rank is not None else "Não informado")
    
    #Pegando as informações de Gênero
    release_a =  movie.findChild("span", {"class":"date"})
    release.append(release_a.text if release_a is not None else "Não informado")

df = pd.DataFrame({
    "Date":release,
    "Title": title_movie,
    "Horary": horary,
    "Language":language,
    "Ranking": sorted(ranking,reverse=True) #Ordenando pela melhor classificação do usuário
})

#Organizando as colunas do DataFrame
df = df[['Date','Title','Horary','Language','Ranking']]                  
df


,Date,Title,Horary,Language,Ranking
0,17 de maio de 2018,Querida Mamãe,19:00,"Legendado,...",Não informado
1,3 de maio de 2018,"Paulo, Apóstolo de Cristo",13:05,"Dublado, 2...","4,1"
2,3 de maio de 2018,Verdade ou Desafio,13:10,"Dublado, 2...","3,6"
3,26 de abril de 2018,Vingadores: Guerra Infinita,13:20,"Dublado, 2...","2,7"
4,19 de abril de 2018,Exorcismos e Demônios,16:20,"Legendado,...","2,5"
5,12 de abril de 2018,Rampage - Destruição Total,13:00,"Dublado, 3...","2,4"
6,5 de abril de 2018,Um Lugar Silencioso,14:10,"Legendado,...","2,4"
7,29 de março de 2018,Nada a Perder - Contra Tudo. Por Todos,17:30,"Legendado,...","2,3"
8,22 de março de 2018,Pedro Coelho,12:50,"Dublado, 2...","1,9"
9,8 de março de 2018,Os Farofeiros,15:00,"Legendado,...","1,5"
